# itch_trade_sign_classification_accuracy_exp

#### Juan Camilo Henao Londono - 07.03.2019
#### AG Guhr - Universitaet Duisburg-Essen

In [1]:
import gzip
import numpy as np
import pandas as pd

In [2]:
ticker = 'AAPL'
year = '2008'
month = '01'
day = '07'

## Trade signs identification

This function returns the theoretical values of time, trade signs, volumes and prices.
With these arrays it is possible to use the models of trade sign classification and test their accuracy

In [3]:
def trade_signs_teo(ticker, year, month, day):
    # Load full data using cols with values time, order, type, shares and price
    data = pd.read_csv(gzip.open('../ITCH_{1}/{1}{2}{3}_{0}.csv.gz'.format(ticker, year, month, day), 'rt'),
                       usecols=(0,2,3,4,5), dtype={'Time': 'uint32', 'Order': 'uint64', 'T': str,
                       'Shares': 'uint16', 'Price': 'float64'})

    data['Price'] = data['Price'] / 10000
    
    # Select only trade orders
    trade_pos = np.array(data['T'] == 'E') + np.array(data['T'] == 'F')
    trade_data = data[trade_pos]
    # Converting the data in numpy arrays
    trade_data_time = trade_data['Time'].values
    trade_data_order = trade_data['Order'].values
    trade_data_types = 3 * np.array(trade_data['T'] == 'E') + 4 * np.array(trade_data['T'] == 'F')
    trade_data_volume = trade_data['Shares'].values
    
    # Select only limit orders
    limit_pos = np.array(data['T'] == 'B') + np.array(data['T'] == 'S')
    limit_data = data[limit_pos]
    # Reduce the values to only the ones that have the same order number
    # as trade orders
    limit_data = limit_data[limit_data.Order.isin(trade_data['Order'])]
    # Converting the data in numpy arrays
    limit_data_order = limit_data['Order'].values
    limit_data_types = 1 * np.array(limit_data['T'] == 'S') - 1 * np.array(limit_data['T'] == 'B')
    limit_data_volume = limit_data['Shares'].values
    limit_data_price = limit_data['Price'].values
    
    # Arrays to store the info of the identified trades
    length_trades = len(trade_data)
    trade_times = 1 * trade_data_time
    trade_signs = np.zeros(length_trades)
    trade_volumes = np.zeros(length_trades, dtype='uint16')
    trade_price = np.zeros(length_trades)
    
    for t_idx in range(len(trade_data)):
        # limit orders that have the same order as the trade order   
        l_idx = np.where(limit_data_order == trade_data_order[t_idx])[0][0]

        # Save values that are independent of the type

        # Price of the trade (Limit data)
        trade_price[t_idx] = limit_data_price[l_idx] 

        # Trade sign identification

        trade = limit_data_types[l_idx]

        if (trade == 1):
            trade_signs[t_idx] = 1.
        else:
            trade_signs[t_idx] = -1.

        # The volume depends on the trade type. If it is 4 the
        # value is taken from the limit data and the order number
        # is deleted from the data. If it is 3 the
        # value is taken from the trade data and then the
        # value of the volume in the limit data must be 
        # reduced with the value of the trade data

        volume_type = trade_data_types[t_idx]

        if (volume_type == 4):

            trade_volumes[t_idx] = limit_data_volume[l_idx]
            limit_data_order[l_idx] = 0

        else:

            trade_volumes[t_idx] = trade_data_volume[t_idx]
            diff_volumes = limit_data_volume[l_idx] - trade_data_volume[t_idx]
            assert diff_volumes > 0

            limit_data_volume[l_idx] = diff_volumes

    assert not sum(trade_signs == 0)
    
    market_time = (trade_times / 3600 / 1000 >= 9.666666) & \
                    (trade_times / 3600 / 1000 < 15.833333)

    trade_times_market = trade_times[market_time] 
    trade_signs_market = trade_signs[market_time]
    trade_volumes_market = trade_volumes[market_time]
    trade_price_market = trade_price[market_time]
    
    return (trade_times_market, trade_signs_market, trade_volumes_market, trade_price_market)

In [4]:
time_teo_ms, trade_sign_teo_ms, volume_teo_ms, price_teo_ms = trade_signs_teo(ticker, year, month, day)

## Equation 2

def itch_taq_trade_signs_consecutive_trades_ms_test(ticker, price_signs,
                                                    trade_signs, times_signs,
                                                    year, month, day):
    """
    Obtain the experimental trade signs based on the change of prices. To
    compute the trades signs are used consecutive trades in the ITCH data.
    Implementatio of the equation (1)
        :param ticker: string of the abbreviation of the stock to be analized
                       (i.e. 'AAPL')
        :param price_signs: price of the trades
        :param trade_signs: theoric trade signs from ITCH data
        :param times_signs: time of the trades
        :param year: string of the year to be analized (i.e '2008')
        :param month: string of the month to be analized (i.e '07')
        :param day: string of the day to be analized (i.e '07')
    """

    print('Accuracy of the trade sign classification for consecutive trades in'
          ' ms for the stock ' + ticker + ' the ' + year + '.' + month + '.'
          + day)

    # trades with values different to zero to obtain the theoretical value
    price_no_0 = price_signs[trade_signs != 0]
    trades_no_0 = trade_signs[trade_signs != 0]
    time_no_0 = times_signs[trade_signs != 0]

    identified_trades = np.zeros(len(time_no_0))

    # Implementation of equation (1). Sign of the price change between
    # consecutive trades
    for t_idx, t_val in enumerate(time_no_0):

        diff = price_no_0[t_idx] - price_no_0[t_idx - 1]

        if (diff):

            identified_trades[t_idx] = np.sign(diff)

        else:

            identified_trades[t_idx] = identified_trades[t_idx - 1]

    assert not len(identified_trades[identified_trades == 0])
    assert len(identified_trades) == len(trades_no_0[trades_no_0 != 0])

    # Accuracy of the classification
    print('For consecutive trades in ms:')
    print('Accuracy of the classification:', round(sum(trades_no_0 == identified_trades) / len(trades_no_0) * 100, 2), '%')
    print('Number of identified trades:', len(trades_no_0))
    print('Number of matches:', sum(trades_no_0 == identified_trades))
    print()
    
    return identified_trades

In [5]:
def itch_taq_trade_signs_consecutive_trades_ms(ticker, price_signs, trade_signs, times_signs, year, month, day):
    
    print('Accuracy of the trade sign classification for the stock ' + ticker + ' the ' + year + '.' + month 
          + '.' + day)
    
    # trades with values different to zero to obtain the theoretical value
    price_no_0 = price_signs[trade_signs != 0]
    trades_no_0 = trade_signs[trade_signs!= 0]
    time_no_0 = times_signs[trade_signs!= 0]
    time_no_0_set = np.array(list(sorted(set(time_no_0))))
    
    identified_trades = np.zeros(len(time_no_0))

    count = 0

    for t_idx, t_val in enumerate(time_no_0_set):

        while (count < len(time_no_0) and time_no_0[count] == t_val):

            diff = price_no_0[count] - price_no_0[count - 1]

            if (diff):

                identified_trades[count] = np.sign(diff)
                count += 1

            else:

                identified_trades[count] = identified_trades[count - 1]
                count += 1
                
    print('For consecutive trades in ms:')
    print('Accuracy of the classification:', round(sum(trades_no_0 == identified_trades) / len(trades_no_0) * 100, 2), '%')
    print('Number of identified trades:', len(trades_no_0))
    print('Number of matches:', sum(trades_no_0 == identified_trades))
    print()
    
    return identified_trades

In [8]:
itch_taq_trade_signs_consecutive_trades_ms(ticker, price_teo_ms,
                                                    trade_sign_teo_ms, time_teo_ms,
                                                    year, month, day)

Accuracy of the trade sign classification for the stock AAPL the 2008.01.07
For consecutive trades in ms:
Accuracy of the classification: 83.03 %
Number of identified trades: 120287
Number of matches: 99871



array([ 1.,  1.,  1., ..., -1., -1.,  1.])